In [1]:
import pandas as pd, numpy as np, json, time
from sklearn.neighbors import KDTree

import sys
sys.path.insert(0,'../../../Architecture/')

from mtree import MTree
from fibheap import *

In [2]:
%%time
df = pd.read_csv('extended_bgp_testbed_5.csv.gz')
df = df.drop(['Unnamed: 0'], axis=1)
df = df.dropna()
df = df.astype('float64')
df.shape

CPU times: user 5.52 s, sys: 232 ms, total: 5.76 s
Wall time: 5.78 s


(185984, 82)

In [3]:
samples_to_use_list = list(range(1000, 10000, 1000))
samples_to_use_list = samples_to_use_list + list(range(10000, 110000, 10000))

In [5]:
results = {}
results['algorithm'] = 'cod'
results['times'] = []

for sample_to_use in samples_to_use_list:
            
    print('cod: {}'.format(sample_to_use))
    results_samples = []
    dfNormalized = df[:sample_to_use]
        
    num_runs = 3
        
    for measurement in range(num_runs):
        sampleSkip = 30
        bufferDF = dfNormalized[0:sampleSkip]
        testDF = dfNormalized[sampleSkip:]

        window_size = 50
        R = 12.5
        k = 5
        
        mtree = MTree()
        data = tuple(map(tuple,testDF.values))

        structure = {}
        indexes = {}

        anomalies = set()
        fibheap = makefheap()  
        
        start = time.time()
        

        for now, instance in enumerate(data):

            if now >= window_size:
                departing_index = now-window_size
                mtree.remove(structure[departing_index]['data'])

                if fibheap.min is None:
                    pass
                else:
                    x = getfheapmin(fibheap)
                    ev = x[0]

                    while (ev==now):
                        x = fheappop(fibheap)

                        index_x = x[1]
                        preceding_objects_x = structure[index_x]['Preceding']
                        if departing_index in preceding_objects_x:
                            preceding_objects_x.remove(departing_index)
                        else:
                            pass

                        if len(preceding_objects) + structure[index_x]['Succeding'] < k:
                            anomalies.add(index_x)
                        else:
                            if len(preceding_objects_x)>0:
                                min_exp = np.inf
                                for preceding_x in preceding_objects_x:
                                    if structure[preceding_x]['exp'] > now:
                                        if structure[preceding_x]['exp'] < min_exp:
                                            min_exp = structure[preceding_x]['exp']
                                            ev = min_exp + window_size + 1
                                if structure[index_x]['exp'] > now:
                                    fheappush(fibheap, (ev, index_x))

                        if fibheap.min is None:
                            break
                        else:
                            x = getfheapmin(fibheap)
                            ev = x[0]


            """
            Generate new entry for current instance
            """
            structure[now] = {
                'Preceding':[],
                'Succeding':0,
                'arr': now,
                'exp': now+window_size,
                'data': instance
            }
            indexes[instance] = now

            """
            Make a range query w.r.t. p. Let A the set of objects returned;
            for each q in A
                nq+ = nq+ 1
            """
            A = mtree.get_nearest(instance, R, k)

            preceding_objects = []
            for q in A:
                q_index = indexes[q[0]]
                structure[q_index]['Succeding'] += 1
                preceding_objects.append(q_index)

                if (q_index in anomalies) and (len(structure[q_index]['Preceding'])+structure[q_index]['Succeding'] >= k):
                    anomalies.remove(q_index)
                    if len(structure[q_index]['Preceding'])!= 0:
                        preceding_q_list = structure[q_index]['Preceding']
                        min_exp = np.inf

                        for preceding_q in preceding_q_list:
                            if structure[preceding_q]['exp'] < min_exp:
                                min_exp = structure[preceding_q]['exp']

                        ev = min_exp + window_size + 1
                        fheappush(fibheap, (ev, q_index))

                else:
                    """
                    Remove from pq object y = min{qi.exp | qi in Pq}
                    """
                    preceding_q_list = structure[q_index]['Preceding']
                    if len(preceding_q_list) > 0:
                        min_exp = np.inf
                        min_index = -1
                        for preceding_q in preceding_q_list:
                            if structure[preceding_q]['exp'] < min_exp:
                                min_exp = structure[preceding_q]['exp']
                                min_index = preceding_q
                        structure[q_index]['Preceding'].remove(min_index)

            structure[now]['Preceding'] = preceding_objects
            if len(preceding_objects) < k:
                """
                Add p to D(R,K)
                """
                anomalies.add(now)
            else:
                """
                ev = min{pi.exp|pi in Pp}
                insert(p, ev+[W/Slide])
                """
                min_exp = np.inf
                for preceding_p in preceding_objects:
                    if structure[preceding_p]['exp'] < min_exp:
                        min_exp = structure[preceding_p]['exp']
                ev = min_exp + window_size + 1
                fheappush(fibheap, (ev, now))
            """
            Add p to data structure supporting range queries
            """
            mtree.add(instance)        
        
        end = time.time()
        time_interval = end-start
        results_samples.append(time_interval)
        print('{} - {}'.format(sample_to_use, time_interval))
    
    results['times'].append(results_samples)
    with open('Results/cod_extended.json', 'w') as f:
        json.dump(results, f, indent=2)

cod: 1000
1000 - 1.7465825080871582
1000 - 1.7604990005493164
1000 - 1.772204875946045
cod: 2000
2000 - 3.651529550552368
2000 - 3.630772113800049
2000 - 3.6346943378448486
cod: 3000
3000 - 5.500812768936157
3000 - 5.531709909439087
3000 - 5.509340524673462
cod: 4000
4000 - 7.36281418800354
4000 - 7.4038612842559814
4000 - 7.374604940414429
cod: 5000
5000 - 9.237375736236572
5000 - 9.260582685470581
5000 - 9.267200708389282
cod: 6000
6000 - 11.142154693603516
6000 - 11.154211282730103
6000 - 11.112234115600586
cod: 7000
7000 - 12.947270154953003
7000 - 12.95736837387085
7000 - 12.980255126953125
cod: 8000
8000 - 14.989800453186035
8000 - 14.88190507888794
8000 - 14.88697361946106
cod: 9000
9000 - 16.807278156280518
9000 - 16.73481011390686
9000 - 16.75785517692566
cod: 10000
10000 - 18.715986013412476
10000 - 18.625662088394165
10000 - 18.679646968841553
cod: 20000
20000 - 37.41167092323303
20000 - 37.42059946060181
20000 - 37.3056538105011
cod: 30000
30000 - 56.614272594451904
30000 -